In [1]:
import pandas as pd;
import numpy as np;

In [2]:
data_frame = pd.read_csv("emails.csv");
# data_frame.to_csv("emails.csv", index = False)
# data_frame = pd.read_csv("emails.csv");

In [3]:
## extracting features from data set.
# X_features = data_frame.iloc[ : , 1: 3001];
# Y = data_frame['Prediction'];

In [4]:
# print(data_frame.shape, X_features.shape);
# print(Y.shape)

In [5]:
data_frame = data_frame.to_numpy();
print(data_frame.shape);

(5172, 3002)


In [6]:
X_features = data_frame[: , 1:3001];
Y = data_frame[: , 3001];

In [7]:
# print(X_features);

In [8]:
## steps in logistic regression
# 1. get the data set
# 2. calculate z  = w*x + b where z is an numpy array
# 3. sigmoid function g(z) = 1 / 1 + e^-z
# 4. write the cost function J(w) = 1/m * summation(-y*(log(a)) - (1 - y)*log(1-a)) where a is sigmoid function and y is the actual output
# 5. write gradient descent algorithm to reduce cost function

In [9]:
def sigmoid(X_train, W, b):
    (n, m) = X_train.shape
    z = np.zeros(n);
    for i in range(n):
        z[i] = np.dot(W, X_train[i]) + b;
    
    a = np.zeros(n);
    
    for i in range(n):
        a[i] = 1 / 1 + np.exp(-z[i])
    
    return a;
    

In [10]:
W = np.random.rand(X_features.shape[1])
y_predict  = sigmoid(X_features, W, 200)

In [40]:
 def cost_func(y_predict, Y):
        '''compute the cost using -ylog(a) - (1 - y)log(1- a)'''
        m = Y.shape[0];
        sm = 0
        for i in range(m):
            sm += (-1 *Y[i]*np.log(y_predict[i])) - ((1 - Y[i])*np.log(1 - y_predict[i]));
        
        #sm = sm / m
        return sm
            

In [39]:
print(cost_func(y_predict, Y));

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
